# Домашнее задание по автобрее №3

Для начала импортируем все, что понадобится.

In [1]:
import re
import spacy
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from tqdm.notebook import tqdm # необяз
import pandas as pd
from pprint import pprint
from collections import Counter
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

Теперь достаем нужный нам датафрейм и чистим имейлы от лишнего.

In [2]:
initial_df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
data = initial_df.content.values.tolist()

# чистим от:
for i, email in enumerate(data):
    # адресов почты
    no_address = re.sub("\S+?@\S+", "", email)
    # переносов и других странных пробелов
    no_n = re.sub("\s+", " ", no_address)
    # от '
    no_sing_quotes = re.sub("\'", " ", no_n)
    data[i] = no_sing_quotes

In [3]:
# результат
data[0]

'From: (where s my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15 I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail. Thanks, - IL ---- brought to you by your neighborhood Lerxst ---- '

Препроцессинг:

In [6]:
# токенизируем
def sent_to_words(sentences):
    for sentence in sentences:
        yield gensim.utils.simple_preprocess(str(sentence), deacc=True)
data_words = list(sent_to_words(data))

# делаем так, чтобы в списке слов выделялись биграммы и триграммы
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
data_words_nostops = remove_stopwords(data_words)
data_words_trigrams = make_trigrams(data_words_nostops)
nlp = spacy.load('en', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_trigrams)

In [8]:
data_lemmatized[0]

['thing',
 'car',
 'nntp_posting_host',
 'line',
 'wonder',
 'could',
 'enlighten',
 'car',
 'see',
 'day',
 'door',
 'sport',
 'car',
 'look',
 'late',
 'early',
 'call',
 'door',
 'really',
 'small',
 'addition',
 'separate',
 'rest',
 'body',
 'know',
 'model',
 'name',
 'engine',
 'spec',
 'year',
 'production',
 'car',
 'make',
 'history',
 'info',
 'funky',
 'look',
 'car',
 'mail',
 'thank',
 'bring',
 'neighborhood',
 'lerxst']

In [9]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1)]]


Оценим разные модели, чтобы найти оптимальное количество топиков. Проверим модель с 5, 10, 15, 20, 25 и 30 топиками.

In [10]:
# функция, оценивающая модель

def evaluate_gensim_model(beg=5, end=30, step=5):
    assert end-beg // step
    
    for i in range(beg, end+1, step):
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=i, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)

        coherence_model = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence = coherence_model.get_coherence()
        print(f'\nNUMBER OF TOPICS: {str(i)}')
        print('\nCoherence Score: ', coherence)

evaluate_gensim_model()


NUMBER OF TOPICS: 5

Coherence Score:  0.5230630234758389

NUMBER OF TOPICS: 10

Coherence Score:  0.4835359582114453

NUMBER OF TOPICS: 15

Coherence Score:  0.5206225931029306

NUMBER OF TOPICS: 20

Coherence Score:  0.48384120995977425

NUMBER OF TOPICS: 25

Coherence Score:  0.4879765099451111

NUMBER OF TOPICS: 30

Coherence Score:  0.41622126834582


Самой coherent оказалась модель с 5 топиками. Строим:

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

У нас получается очень странный первый топик, но мы следуем заданию: самая coherent модель -- лучшая.

In [50]:
lda_model.show_topics(formatted=False)

[(0,
  [('ax', 0.6231141),
   ('di_di_di_di', 0.0057803304),
   ('lq', 0.0024715352),
   ('dy', 0.0023401112),
   ('rlk', 0.0021645264),
   ('oo', 0.0015877903),
   ('gainey', 0.0015189296),
   ('chz', 0.001483495),
   ('tdo', 0.0011262225),
   ('tq', 0.0010716837)]),
 (1,
  [('would', 0.016752828),
   ('write', 0.014647075),
   ('article', 0.010235661),
   ('think', 0.009491588),
   ('make', 0.009012101),
   ('line', 0.008514239),
   ('people', 0.008439936),
   ('may', 0.008355028),
   ('know', 0.007803973),
   ('say', 0.0071977074)]),
 (2,
  [('go', 0.016044628),
   ('line', 0.013591605),
   ('year', 0.012195823),
   ('get', 0.010615865),
   ('write', 0.008980099),
   ('team', 0.008316611),
   ('car', 0.008276554),
   ('good', 0.007835208),
   ('game', 0.007493373),
   ('see', 0.0068138028)]),
 (3,
  [('say', 0.013462576),
   ('people', 0.013290287),
   ('evidence', 0.012013258),
   ('believe', 0.008645084),
   ('reason', 0.0076174526),
   ('man', 0.0070818453),
   ('law', 0.00665786

Маллет не работает из-за джавы((

Чтобы код прошелся по топикам быстрее и быстрее определил главный топик, создадим словарь топиков. топик: {слово: вес}

In [76]:
topic_words = {}
for topic in lda_model.print_topics():
    wordlist = topic[1].split(" + ")
    wrds_and_wghts = {}
    for wrd in wordlist:
        wrds_and_wghts[wrd.split("*")[1].strip('"')] = float(wrd.split("*")[0])
    topic_words[topic[0]] = wrds_and_wghts

Функция для определения основного топика:

In [77]:
def main_topic(lemmas):
    topic_counter = {i: 0 for i in range(20)}
    for lemma in lemmas:
        for topic in topic_words:
            if lemma in topic_words[topic]:
                topic_counter[topic] += topic_words[topic][lemma]
    return max(topic_counter, key=topic_counter.get)

In [78]:
main_topic(["people", "write", "article"])

1

Создаем датафрейм из лемматизированных текстов:

In [84]:
df = pd.DataFrame(data=[" ".join(text) for text in data_lemmatized], columns=["text"])

Прибавляем к датафрейму соответствующие номера топиков:

In [85]:
topics_df = []
for text in data_lemmatized:
    topics_df.append(main_topic(text))
df.insert(1, "topic", topics_df)

In [86]:
df[:5]

,text,topic
0,thing car nntp_posting_host line wonder could ...,2
1,si poll final summary final call si clock repo...,4
2,question organization purdue_university_engine...,2
3,division line amber write write article know c...,2
4,question organization smithsonian_astrophysica...,1


Теперь TF-IDF. Создаем пять групп, чтобы внутри каждой можно было считать свои TF-IDFы:

In [57]:
data0 = list(df['text'][df.topic == 0])
data1 = list(df['text'][df.topic == 1])
data2 = list(df['text'][df.topic == 2])
data3 = list(df['text'][df.topic == 3])
data4 = list(df['text'][df.topic == 4])
# объединим их для удобного цикла
corps = (data0, data1, data2, data3, data4)

Здесь лучше всего взять sklearn и через его функции посчитать TF-IDFы. Создаем список tf_idf_list_all_data, который будет состоять из списков (tf_idf_list), содержащих в себе по пять самых больших TF-IDFов на текст.

In [59]:
tf_idf_list_all_data = []
for corp in corps:
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(corp)
    tf_idf_list = []
    for text_tfidf in tqdm(tfIdf):
        df_tf_idf = pd.DataFrame(text_tfidf.T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
        df_tf_idf = df_tf_idf.sort_values('TF-IDF', ascending=False)[:5]
        tf_idf_list.append(list(df_tf_idf.to_records()))
    tf_idf_list_all_data.append(tf_idf_list)

Вот, к примеру, TF-IDFы всех текстов первого топика:

In [60]:
tf_idf_list_all_data[1]

[[('error', 0.41612475),
  ('bug', 0.26551402),
  ('warn', 0.24552914),
  ('memory', 0.21081822),
  ('smithsonian_astrophysical_observatory', 0.19786584)],
 [('weapon', 0.65797943),
  ('mass_destruction', 0.2959507),
  ('needless', 0.15886762),
  ('term', 0.14797465),
  ('individual', 0.14726251)],
 [('scsi', 0.77121793),
  ('fast', 0.24611016),
  ('scsi_controller_chip', 0.21731245),
  ('range', 0.21690588),
  ('ide', 0.16613159)],
 [('icon', 0.6491346),
  ('wallpaper', 0.38919873),
  ('download', 0.33288709),
  ('help', 0.2985444),
  ('win', 0.21625071)],
 [('board', 0.56126453),
  ('stac', 0.28474897),
  ('sigma_design', 0.2718567),
  ('licensing', 0.22287632),
  ('icon', 0.21316633)],
 [('parent', 0.38559093),
  ('child', 0.29090283),
  ('swear', 0.28733161),
  ('multiple_moral_code', 0.23959506),
  ('moral', 0.21478262)],
 [('trial', 0.31970161),
  ('holocost', 0.18732319),
  ('zyklon', 0.18732319),
  ('exterminated', 0.18732319),
  ('mutilated', 0.18732319)],
 [('thermocouple', 0

Создаем цикл, который соединит все датафреймы по разным текстам с разными топиками:

In [90]:
df = pd.DataFrame(data=data0, columns=["text"])
df.insert(1, "topic", [0]*len(df))
df.insert(2, "tf_idfs", tf_idf_list_all_data[0])
for i, corp in enumerate(corps[1:]):
    df_add = pd.DataFrame(data=corp, columns=["text"])
    df_add.insert(1, "topic", [i+1]*len(df_add))
    df_add.insert(2, "tf_idfs", tf_idf_list_all_data[i+1])
    df = df.append(df_add)

Получаем такую таблицу, что мы и хотели!

In [91]:
df

,text,topic,tf_idfs
0,hit card cost street price,0,"[[street, 0.510563268278221], [hit, 0.46566643..."
1,ax ax ax ax ax ax ax ax ax ax ax ax ax wwiz_bh...,0,"[[ax, 0.9999727224420126], [di_di_di_di, 0.002..."
2,happy birthday birthday,0,"[[birthday, 0.8944271909999159], [happy, 0.447..."
3,look test,0,"[[test, 0.7534022658171431], [look, 0.65755990..."
4,,0,"[[aao, 0.0], [premature, 0.0], [precipitate, 0..."
...,...,...,...
2970,use line nntp_posting_host small problem pro g...,4,"[[dma, 0.471047710724459], [pro, 0.39264402947..."
2971,production company research line machine sale ...,4,"[[memory, 0.35354033863133955], [song, 0.34121..."
2972,organization reilly line reply thank many offe...,4,"[[review, 0.5193951565321043], [reilly, 0.2976..."
2973,problem screen blank sometimes minor physical ...,4,"[[blank, 0.3577574415290302], [wire, 0.3178574..."


Кратко про coherence: конкретно в этой работе мы использовали c_v -- один из способов вычислить coherence модели. C_v берет окно из слов в тексте и смотрит, появляются ли в окне рядом слова из одного топика. Между всеми главным словами топика c_v вычисляет NPMI (число от 1 до -1, обозначающее частоту нахождения двух слов рядом друг с другом), и для каждого слова строит вектор из полученных NPMI. Вычисляется косинусное расстояние между каждым вектором и суммой векторов. Среднее арифм. косинусного расстояния и есть coherence модели.

*Миша Сонькин*